In [1]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [2]:
user_model = BayesianModel([('R', 'I'),
                               ('R', 'G'),
                               ('R', 'H'),
                               ('R', 'K'),
                               ('H','K') 
                           ])

<img src="./tujiegou.png"  height="330" width="495">
<img src="./biao123.png"  height="530" width="895">
<img src="./biao45.png"  height="530" width="895">


In [7]:
fannum_cpd = TabularCPD( 
    variable='K', # 节点名称    
    variable_card=2, # 节点取值个数   
    values=[
                   [0.89,0.01,0.6,0.9], # 该节点的概率表    
                   [0.11,0.99,0.4,0.1]
                   ] ,
                 evidence=['R', 'H'], # 该节点的依赖节点 
                evidence_card=[2, 2] # 依赖节点的取值个数    
)   
 
    
account_cpd = TabularCPD(    
            variable='R',   
            variable_card=2,    
            values=[[0.4, 0.6]] 
)   
 
    
header_cpd = TabularCPD( 
            variable='H',   
            variable_card=2,
            values=[
                    [0.9,0.2],
                    [0.1,0.8]
                   ],
            evidence=['R'], 
            evidence_card=[2] 
)   
 
    
dynamic_cpd = TabularCPD(    
            variable='G',   
            variable_card=2,    
            values=[
                    [0.82,0.1],   
                    [0.18,0.9]
                   ],  
            evidence=['R'], 
            evidence_card=[2]   
)   
 
    
info_cpd = TabularCPD(   
            variable='I',   
            variable_card=2,    
            values=[
                    [0.75, 0.3],    
                    [0.25, 0.7]
                   ],   
            evidence=['R'], 
            evidence_card=[2]   
)

In [8]:
user_model.add_cpds(
                    account_cpd,
                    info_cpd,
                    dynamic_cpd,
                    header_cpd,
                    fannum_cpd
                    )

In [15]:
from pgmpy.inference import VariableElimination 
user_infer = VariableElimination(user_model)  
 
    
prob_K = user_infer.query(   
                          variables=['K'],    
                          evidence={'R': 0}
                        )  
print(prob_K)

Eliminating: H: 100%|██████████| 3/3 [00:00<00:00, 640.03it/s]

+------+----------+
| K    |   phi(K) |
+======+==========+
| K(0) |   0.8020 |
+------+----------+
| K(1) |   0.1980 |
+------+----------+


In [10]:
# 生成数据  
import numpy as np  
import pandas as pd 
 
    
raw_data = np.random.randint(low=0, high=2, size=(1000, 5)) 
data = pd.DataFrame(raw_data, columns=['R', 'I', 'G', 'K', 'H'])    
data.head()

,R,I,G,K,H
0,1,0,0,0,1
1,1,1,0,1,0
2,0,1,0,0,1
3,0,1,1,0,0
4,0,1,1,0,1


In [13]:
# 定义模型  
from pgmpy.models import BayesianModel  
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator  
    
model = BayesianModel([('R', 'I'), ('R', 'G'), ('R', 'K'), ('R', 'H'),('H','K')]) 
    
# 基于极大似然估计进行模型训练    
model.fit(data, estimator=MaximumLikelihoodEstimator)   

for cpd in model.get_cpds():    
    # 打印条件概率分布  
    print("CPD of {variable}:".format(variable=cpd.variable))   
    print(cpd)

CPD of G:
+------+---------------------+--------------------+
| R    | R(0)                | R(1)               |
+------+---------------------+--------------------+
| G(0) | 0.48893360160965793 | 0.4711729622266402 |
+------+---------------------+--------------------+
| G(1) | 0.5110663983903421  | 0.5288270377733598 |
+------+---------------------+--------------------+
CPD of H:
+------+---------------------+---------------------+
| R    | R(0)                | R(1)                |
+------+---------------------+---------------------+
| H(0) | 0.545271629778672   | 0.5248508946322068  |
+------+---------------------+---------------------+
| H(1) | 0.45472837022132795 | 0.47514910536779326 |
+------+---------------------+---------------------+
CPD of I:
+------+---------------------+---------------------+
| R    | R(0)                | R(1)                |
+------+---------------------+---------------------+
| I(0) | 0.5070422535211268  | 0.5228628230616302  |
+------+---------------